In [ ]:
def get_current_peer_with_role(rolename):    
    if rolename == 'Buyer':
        return peer_buyer
    elif  rolename == 'Seller':   
        return peer_owner
    elif  rolename == 'Factor':   
        return peer_factor
    elif  rolename == 'TaxInspector':    
        return peer_taxinspector    

    
def reload_clicked1(i):
    global buyer,peer_buyer
    global asset
    
    user = get_current_user_with_role(roleOptions.value)
    peer = get_current_peer_with_role(roleOptions.value)
    
    asset = listInvoice(peer, id, user=user)
    
    with _out:
        buyer=get_cert_cn(asset['buyer'])
        peer_buyer=get_peer_of_user(buyer)
        
    widget_load_asset(_widget,asset)
    role_selected('')  
 
def _block_btn_reload_clicked1(i):    
    height=get_height()
    _block_text.value = str(height)
    with _block_out:
        clear_output()  
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        _t = get_transaction()[0]
        clear_output()  
        facturx_widgets(_t,'/', hideUnused=True, hide=False).output
        
       


# invoiceDemoLibrary

In [1]:
def calc_keys(assets):
    _keys = []
    for asset in assets:
        _keys.append(asset['Key'])
    return _keys    

def getUsedRoles(roles):
    _all=[]
    for element in roles:
        for role in element['roles']:
            if role not in _all:
                _all.append(role)
    return _all

# Start

In [2]:
from ipynb.fs.full.facturxWidget import *
#from ipynb.fs.full.kubernetesTool import *
#from ipynb.fs.full.invoiceSmartcontract import *

from ipynb.fs.full.blockchain import *

In [13]:
_blockchain = Blockchain(channel_name='invoicing25',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

In [14]:
_blockchain.installWalletTool()


In [15]:
_blockchain.peers['aubonmoulin'].installWalletTool(sharepoint=_blockchain.sharepoint,channel_name=_blockchain.channel_name)

In [16]:
_peer = _blockchain.peers['aubonmoulin']

In [25]:
_user = User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)

In [18]:
roles=_user.smart_contract.getAllRoles()[0]
roleTransactions=_user.smart_contract.getRoleTransactions()[0]
usedRoles=getUsedRoles(roles)

In [28]:
_user.smart_contract.getRoles()[0]['taxInspector']

'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'

In [19]:
_user.smart_contract.getRoles()[0]

{'name': 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR',
 'roles': ['Seller'],
 'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'}

In [20]:
User(peer=_blockchain.peers['megustaolive'], name='Pedro.Sanchez@megustaolive.es',channel_name=_blockchain.channel_name).smart_contract.getRoles()

({'name': 'x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
  'roles': ['Seller', 'Buyer'],
  'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'},
 Output())

In [21]:
#User(peer=_blockchain.peers['aubonmoulin'], name='Stephanie.Hoarau@hotelsaintdenis.re', channel_name=_blockchain.channel_name).smart_contract.listInvoices()
assets = _user.smart_contract.listInvoices()[0]
_keys = calc_keys(assets)

{'peer': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7f4590a4bf40>,
 'name': 'Stephanie.Hoarau@hotelsaintdenis.re',
 'smart_contract': <ipynb.fs.full.invoiceSmartcontract.InvoiceSmartContract at 0x7f4590af6250>}

In [ ]:
_org = _cn.split('@')[1]
for key,peer in _blockchain.peers.items():
    if _org in peer.organizations.keys():
        if _user in peer.organizations[_org].users.keys():
            peer.organizations[_org].users[_user]            

In [ ]:
for key,peer in _blockchain.peers.items():
    print(peer.organizations.keys())

### old

In [ ]:
import ipywidgets as widgets
_out = widgets.Output()
with _out:
    from ipynb.fs.full.invoiceDemoLibrary import *

In [ ]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]
_keys = calc_keys(assets)

# Demo Functions

In [ ]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [215]:
def get_current_user_with_role(rolename,asset):    
    if rolename == 'Buyer':
        _cn = get_cert_cn(asset['buyer'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Seller':  
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'Factor':   
        _cn = get_cert_cn(asset['owner'])
        return _blockchain.getUser(_cn)
    elif  rolename == 'TaxInspector':    
        _cn = get_cert_cn(asset['owner'])
        _owner = _blockchain.getUser(_cn)                    
        _cn = get_cert_cn(_owner.smart_contract.getRoles()[0]['taxInspector'])
        return _blockchain.getUser(_cn)

### Widgets

In [327]:
import functools

class InvoiceWidget(object):
    def __init__(self,user=None,usedRoles=None,asset_keys=None):
        
        self.roleOptions = widgets.Dropdown(options=usedRoles)
        self.roleOptions.observe(self.reload_clicked, names='value')

        self.btn = widgets.Button(description=f'From Blockchain', disabled=False,button_style='success', icon='check')    
        self.btn.on_click(self.reload_clicked)

        self.assetOptions = widgets.Dropdown(options=asset_keys)
        self.assetOptions.observe(self.asset_selected, names='value')
        self.options = widgets.VBox([self.assetOptions, self.roleOptions])

        # explore blocks
        self.block_btn = widgets.Button(description=f'Explore Block', disabled=False,button_style='success', icon='check')    
        self.block_btn.on_click(self.block_btn_reload_clicked)
        self.block_text = widgets.Text(description = 'height', style=style, layout=layout)
        
        self.block_text.on_submit(self.on_value_submit_block_text)   
    
        self.block_out = widgets.Output()
        self.block_explorer = widgets.VBox([self.block_btn,self.block_text,self.block_out])                

        self.assetOptions.value='asset14'    
        self.user  = user
        self.asset = self.user.smart_contract.listInvoice(self.assetOptions.value)[0]
        
        self.widget = facturx_widgets(asset,'/', hideUnused=False, hide=True) 
        self.widget_vbox = widgets.VBox([self.btn,self.widget.output])
        
        self.HBox   = widgets.HBox([self.widget_vbox, self.options,self.block_explorer]) 
        self.enable_fields_for_role()
        
        # fix empty header line
        self.widget.container.fields[' '].widget.layout=widgets.Layout(display='flex',height='5px')
        box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='24%')
        self.widget_vbox.layout=box_layout 
        box_layout2 = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='flex-end',
                width='34%')
        self.block_explorer.layout=box_layout2 
        display(self.HBox)
        
        #self.reload_clicked('')
        
    def widget_load_asset(self,asset):
        self.widget.output.clear_output()
        self.widget.container = FieldContainer()
        with self.widget.output:
            self.widget.addChilds(asset,'/', hideUnused=True)

    def reload_clicked(self,i):
        id=self.assetOptions.value
        self.user  = get_current_user_with_role(self.roleOptions.value, self.asset)
        self.asset = self.user.smart_contract.listInvoice(id)[0]
        self.widget_load_asset(self.asset)
        self.init_fields_with_role(self.user,id) 
        
        # fix empty header line
        self.widget.container.fields[' '].widget.layout=widgets.Layout(display='flex',height='5px')

    def role_selected(self,x): 
        self.user = get_current_user_with_role(roleOptions.value,asset)
        self.init_fields_with_role(self.user) 
        
    def init_fields_with_role(self,user,id):    
        if roleOptions.value == 'Buyer':
            self.init_fields_buyer(user=user,id=id)
        elif roleOptions.value == 'Seller':
            self.init_fields_seller(user=user,id=id)
        elif roleOptions.value == 'TaxInspector':    
            self.init_fields_taxinspector(user=user,id=id)
        elif roleOptions.value == 'Factor':    
            self.init_fields_factor(user=user,id=id)

    def asset_selected(self,x): 
        self.reload_clicked('')

    def on_value_submit_block_text(self,change): 
        self.update_block_explorer(change.value)

    def update_block_explorer(self,id):
        _id = int(id) - 1
        Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).uploadBlockToShare(id=_id)
        _block=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).getBlockFromShare(id=_id)
        with self.block_out:  
            clear_output()  
            _t = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block)
            facturx_widgets(_t,'/', hideUnused=True, hide=False).output
            _b = Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_data(block = _block)
            facturx_widgets(_b,'/', hideUnused=True, hide=False).output        
        
    # !!! depends on peer, channel     
    def block_btn_reload_clicked(self,i):   
        height=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).get_height()
        self.block_text.value = str(height)
        self.update_block_explorer(height)
        
            
    def disable_all_fields(self):
        for key,value in self.widget.container.fields.items():
            value.widget.disabled=True

    def enable_fields(self,fieldnames):
        for fieldname in fieldnames:
            self.widget.container.fields[fieldname].widget.disabled=False
            
    def enable_fields_for_role(self):
        rolename=self.roleOptions.value
        self.disable_all_fields()
        if rolename == 'Buyer':
            self.enable_fields(['received','receivedOrder'])
        elif  rolename == 'Seller':   
            self.enable_fields(['sold','claimPaid','claimPaidBy','owner'])
        elif  rolename == 'Factor':   
            self.enable_fields(['claimPaid','claimPaidBy'])
        elif  rolename == 'TaxInspector':   
            self.enable_fields(['taxReceived'])    
            
            
    def init_fields_buyer(self,user=None,id=None):
    
        def received_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedInvoice(id=id)[1]

        def receivedOrder_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedOrder(id=id)[1]    

        self.widget.container.fields['received'].widget.observe(functools.partial(received_changed,user=user,id=id))
        self.widget.container.fields['receivedOrder'].widget.observe(functools.partial(receivedOrder_changed,user=user,id=id))
        self.enable_fields_for_role()


    def init_fields_seller(self,user=None,id=None):

        def sold_changed(b,user=None,id=None):
            if b['name'] == 'value':
                name=_widget.container.fields['owner'].widget.value
                user.smart_contract.transferInvoice(id=id,name=name)[1]

        def claimPaid_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedPayment(id=id, payer= _widget.container.fields['claimPaidBy'].widget.value)

        self.enable_fields_for_role()    
        self.widget.container.fields['sold'].widget.observe(functools.partial(sold_changed,user=user,id=id))
        self.widget.container.fields['claimPaid'].widget.observe(functools.partial(claimPaid_changed,user=user,id=id))  
        self.replace_owner_field_by_dropdown()


    def init_fields_factor(self,user=None,id=None):

        def claimPaid_changed(b,user=None,id=None):
            if b['name'] == 'value':
                user.smart_contract.receivedPayment(id=id, payer= _widget.container.fields['claimPaidBy'].widget.value)

        self.enable_fields_for_role()    
        self.widget.container.fields['claimPaid'].widget.observe(functools.partial(claimPaid_changed,user=user,id=id))  


    def init_fields_taxinspector(self,user=None,id=id):

        def taxReceived_changed(b,user=None,id=None):
            if b['name'] == 'value':
                 user.smart_contract.taxReceived(id=id)

        self.widget.container.fields['taxReceived'].widget.observe(functools.partial(taxReceived_changed,user=user,id=id))
        self.enable_fields_for_role()    

    # !!! roles
    def create_dropdown_owner(self):
        _certs=[self.widget.container.fields['owner'].widget.value]
        _certs = _certs + get_person_certs_with_role('Factor',roles)
        return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

    def replace_owner_field_by_dropdown(self):
        _owners = self.create_dropdown_owner()
        self.widget.container.fields['owner'] = Field(widget=_owners,path=self.widget.container.fields['owner'].path)
        self.widget.redisplay()    

In [328]:
_widget=InvoiceWidget(user=_user,usedRoles=usedRoles,asset_keys=_keys)

In [ ]:
def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)
    elif roleOptions.value == 'Factor':    
        init_fields_factor(user=factor,peer=peer_factor)

In [ ]:
def create_address_widget(_invoice):
    _widget1 =  widgets.HBox([widgets.VBox([facturx_widgets(_invoice,'/exchangedDocumentContext', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/exchangedDocument', hideUnused=True, hide=True).output]) ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', hideUnused=True, hide=True).output])
    #display (_widget1)
    return _widget1


def create_pod_widget(_invoice,blockchain):
    pod_list = []
    for key,peer in blockchain.peers.items():
        pod_list.append(facturx_widgets(peer.__dict__,'/', hideUnused=True, hide=True).output)
    _widget = widgets.VBox(pod_list)
    return _widget

def create_cluster_widget():
    output = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'get','service','hl-fabric-cc-external-invoice-net','-o','json'])
    _chaincode_service = json.loads(output.decode('ascii'))
    _chaincode_service_fields = {'name':_chaincode_service['metadata']['name'], 'nodePort':_chaincode_service['spec']['ports'][0]['nodePort']}

    _chaincode_widget = facturx_widgets(hideUnused=True, hide=True)
    #_header = _chaincode_widget.addHeader(text='Chaincode',bold=True, path='',hide=True )
    #_chaincode_widget.container.add(widget=_header)
    _chaincode_widget.append(data=_chaincode_service_fields)
    
    shared_fs_host=cc.shared_fs
    _sharepoint_widget = facturx_widgets({'sharePoint':shared_fs_host},'/', hideUnused=True, hide=True).output

    _peer_pods = create_pod_widget(_invoice,_blockchain)
    _orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer').__dict__
    _orderer_pod = facturx_widgets(_orderer,'/', hideUnused=True, hide=True).output

    _column2 = widgets.VBox([_orderer_pod, _chaincode_widget.output, _sharepoint_widget])
    #_chaincode_widget.redisplay()
    return widgets.HBox([_peer_pods, _column2])


def display_role(roles, role_transctions, role,user,peer):
    _role_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '390px'}, style = {'description_width': '110px'})
    _header = _role_widget.addHeader(text='Rolle',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data=roles) 
    _header = _role_widget.addHeader(text='User',bold=True, path='',hide=True )
    _role_widget.container.add(widget=_header)
    _role_widget.append(data={'user':user,'peer':peer.name})
    
    _role_transctions_widget = facturx_widgets( hideUnused=True, hide=True, layout = {'width': '300px'}, style = {'description_width': '50px'})
    _header = _role_transctions_widget.addHeader(text='Transaktionen: ' + role,bold=True, path='',hide=True  )
    _role_transctions_widget.container.add(widget=_header)
    _role_transctions_widget.append(data=role_transctions)

    _hbox = widgets.HBox([_role_widget.output,_role_transctions_widget.output])
    display(_hbox)
    _role_widget.redisplay()
    _role_transctions_widget.redisplay()

In [ ]:
def get_transactions_by_role(role):
    for _role in roleTransactions:
        if _role['role'] == role:
            return _role['transactions']

def update_role_widget(user=None,peer=None,role=None):
    _roles = getRoles(peer, user=user)[0]
    _role_transactions = get_transactions_by_role(role)
    
    if isinstance(_roles, str):
        _roles=''
    
    with roleWidget_out:
        clear_output()
        display_role(_roles, _role_transactions,role,user,peer)

def role_widget_selected(x): 
    if roleWidgetOptions.value == 'Buyer':
        update_role_widget(user=buyer,peer=peer_buyer,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'Seller':
        update_role_widget(user=owner,peer=peer_owner,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'TaxInspector':    
        update_role_widget(user=taxinspector,peer=peer_taxinspector,role=roleWidgetOptions.value)
    elif roleWidgetOptions.value == 'Factor':    
        update_role_widget(user=factor,peer=peer_factor,role=roleWidgetOptions.value)                     
        

roleWidgetOptions = widgets.Dropdown(options=usedRoles)
roleWidgetOptions.observe(role_widget_selected, names='value')     
roleWidget_out = widgets.Output()

In [ ]:
display(roleWidgetOptions,roleWidget_out)    

# Blockchain (Peers, Orderer, Chaincode) und Sharepoint (WebDAV)

In [ ]:
cluster_widget = create_cluster_widget()
display(cluster_widget)

# Rechnung aus Github-Repository (Factur-X / Parquet)

In [ ]:
facturx_widgets(data=_invoice,path='/supplyChainTradeTransaction', hideUnused=True, hide=True).output

# Adressen

In [ ]:
create_address_widget(_invoice)

# Rollen (werden in Blockchain gespeichert und sind änderbar)

In [ ]:
display(roleWidgetOptions,roleWidget_out)    

# Invoice Widget

In [178]:
 asset = _user.smart_contract.listInvoice('asset14')[0]
_widget = create_invoice_widget(asset)
asset_selected('')

In [158]:
_out

Output(outputs=({'name': 'stderr', 'text': '\x1b2022-03-02 15:47:49.480 UTC [channelCmd] InitCmdFactory ->…

In [155]:
_block_btn_reload_clicked('')

2022-03-02 15:41:53.855 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2022-03-02 15:41:53.858 UTC [cli.common] readBlock -> INFO 002 Received block: 67
2022-03-02 15:41:53.890 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2022-03-02 15:41:53.890 UTC [common.tools.configtxgen] doInspectBlock -> INFO 002 Inspecting block
2022-03-02 15:41:53.890 UTC [common.tools.configtxgen] doInspectBlock -> INFO 003 Parsing genesis block
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   230  100   230    0     0  32857      0 --:--:-- --:--:-- --:--:-- 32857
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   242  100   242    0     0  80666      0 --:--:-- --:--:-- --:--:-- 80666
  % Total    % Received % Xferd  A

In [127]:

asset = _user.smart_contract.listInvoice(id)[0]
asset
_user.smart_contract.listInvoice(id)[1]

Output()

In [133]:
_cn = get_cert_cn(_user.smart_contract.getRoles()[0]['taxInspector'])
asset=_blockchain.getUser(_cn).smart_contract.listInvoice(id)[0]

In [152]:
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).get_height()

67

In [175]:
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block2)

['TaxReceived', 'asset6']

In [173]:
_block=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).getBlockFromShare()
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block=_block)

['TaxReceived', 'asset6']

In [322]:
#_cn = get_cert_cn(asset['owner'])
#asset
#get_height()
#_out

#Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).uploadBlockToShare()
_block=Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).getBlockFromShare()
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block)
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_data(block = _block)

{'ID': 'asset6',
 'owner': 'x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
 'buyer': 'x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR',
 'hash': -6366548217084274917,
 'invoiceNumber': 'FA-2017-0010',
 'tax': 46.25,
 'netto': 624.9,
 'countryOrigin': 'FR',
 'countryBuyer': 'FR',
 'status': '',
 'received': True,
 'receivedOrder': True,
 'sold': True,
 'claimPaid': True,
 'claimPaidBy': '',
 'taxExemptionReason': '',
 'taxReceived': True}

In [117]:
Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_data(block = _block)

Block(peer=_blockchain.peers['aubonmoulin'],channel_name=_blockchain.channel_name,sharepoint=SharePoint()).decode_block_transaction(block = _block)

['ReceivedPayment', 'asset6', '']

In [80]:
_blockchain.peers

{'aubonmoulin': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7f4590a4bf40>,
 'lyon-impots': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7f4590a4bdc0>,
 'austriafactor': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7f4590af6850>,
 'megustaolive': <ipynb.fs.full.blockchain.KubernetesPeer at 0x7f4590af68e0>}

In [ ]:
height=get_height()
_block_text.value = str(height)
uploadBlockToShare(height-1)    
_block = getBlockFromShare(height-1)
_invoice = decode_block_invoice(_block)
_transaction = decode_block_transaction(_block)
with _block_out:
    clear_output()  
    facturx_widgets(_transaction,'/', hideUnused=True, hide=False).output

In [ ]:
_transaction, _invoice

In [ ]:
with _block_out:
    clear_output()  

height=get_height()
_block_text.value = str(height)    